In [ ]:
# pip install boto3
import boto3, time # see https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/lightsail.html 

# You input is necessary

* Choose region_name
* paste aws id, aws key and domainName
* Change domain registration according to the name servers listed in the end

In [ ]:
region_name='us-east-1' 
# us-east-2 @ Ohio
# us-east-1 @ N. Virginia
# us-west-2 @ Oregon
# ap-south-1 @ Mumbai
# ap-northeast-2 @ Seoul
# ap-southeast-1 @ Singapore
# ap-southeast-2 @ Sydney
# ap-northeast-1 @ Tokyo
# ca-central-1 @ Central
# eu-central-1 @ Frankfurt
# eu-west-1 @ Ireland
# eu-west-2 @ London
# eu-west-3 @ Paris

In [ ]:
##### Learn how to get these info
##### https://docs.aws.amazon.com/powershell/latest/userguide/pstools-appendix-sign-up.html
aws_access_key_id='AKIAMMLKFNWRZDLRJ' 
aws_secret_access_key='97i8FNOUEIBw3b1/VEUb9w31/014rQ3egBJN' 

In [ ]:
domainName = 'your.domain.name'

# Operations

### Setup Client for the region of N. Virginia

In [ ]:
client = boto3.client('lightsail', region_name=region_name, use_ssl=True, 
                      aws_access_key_id=aws_access_key_id, 
                      aws_secret_access_key=aws_secret_access_key)

### Create Instance in N. Virginia

In [ ]:
response = client.create_instances(
    instanceNames=['created_from_python',],
    availabilityZone=region_name+'a',
    blueprintId='ubuntu_18_04',
    bundleId='nano_2_0', #  micro_2_0
    userData='echo "success">/home/ubuntu/test.log', # Initialization shellscript
) 

while 1:
    response = client.get_instance_state(instanceName='created_from_python')
    if response['state']['name'] == 'running': 
        print('New Instance is ready~')
        break
    time.sleep(5)

### Allocate static ip

In [ ]:
response = client.allocate_static_ip(staticIpName='static_ip_from_python')

### Attach static ip to instance

In [ ]:
response = client.attach_static_ip(
    staticIpName='static_ip_from_python',
    instanceName='created_from_python'
)

##### Get Static ip

In [ ]:
response = client.get_static_ip(
    staticIpName = 'static_ip_from_python'
)
static_ip = response['staticIp']['ipAddress']
print(static_ip)

### Open port on Firewall 

In [ ]:
response = client.open_instance_public_ports(
    portInfo={
        'fromPort': 443,
        'toPort': 443,
        'protocol': 'tcp',
        'cidrs': [ "0.0.0.0/0", ],
    },
    instanceName='created_from_python'
)

##### Get 'A' DNS record

In [ ]:
A_id = [i['id'] for i in client.get_domain( domainName=domainName)['domain']['domainEntries'] if i['type'] == 'A'][0]

### Change DNS recordset

In [ ]:
response = client.update_domain_entry(
    domainName=domainName,
    domainEntry={
        'id': A_id,
        'name': domainName,
        'target': static_ip, 
        'type': 'A', 
    }
)

### Delete Instance

In [ ]:
response = client.delete_instance(
    instanceName='created_from_python', 
)

### Delete static ip

In [ ]:
response = client.release_static_ip(
    staticIpName='static_ip_from_python'
)

### Create attachable disk

In [ ]:
response = client.create_disk(
    diskName='create_disk_from_python',
    availabilityZone=region_name+'a',
    sizeInGb=8, 
)

### Attach it

In [ ]:
response = client.attach_disk(
    diskName='create_disk_from_python',
    instanceName='created_from_python',
    diskPath='/dev/xvdx'
)

### Partition and Formatting

In [ ]:
sudo parted /dev/xvdx mklabel gpt mkpart part_label_1 ext4 1MiB 7.99GiB
sudo mkfs.ext4 /dev/xvdx1

### Start instance

In [ ]:
response = client.start_instance(
    instanceName='created_from_python'
)

while 1:
    response = client.get_instance_state(instanceName='created_from_python')
    if response['state']['name'] == 'running': 
        print('New Instance is ready~')
        break
    time.sleep(5)

### Stop instance

In [ ]:
response = client.stop_instance(
    instanceName='created_from_python'
)

while 1:
    response = client.get_instance_state(instanceName='created_from_python')
    if response['state']['name'] == 'stopped': 
        print('Instance stopped~')
        break
    time.sleep(5)